<a href="https://colab.research.google.com/github/lenchanti/TMD-framework/blob/main/unimodel_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import re
import numpy as np
from scipy import interpolate
import pandas as pd
#import tensorflow_decision_forests as tfdf
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn import tree
from glob import glob
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

path="/content/drive/MyDrive/merge"


# z-변환 함수 정의
def z_normalize(data):
    return (data - np.mean(data)) / np.std(data)


ori_df=pd.DataFrame()
mode=pd.DataFrame()
for sample_files in os.listdir(path):

    sample_file=pd.read_csv(path+"//"+sample_files)

    #X=sample_file.iloc[:,:-2]
    # y=sample_file[['Mode', 'Survay1']]
    sample_mode_file=sample_file[['Mode']]
    # sample_mode_file=pd.get_dummies(sample_mode_file,columns=["Mode"])
    col_names=sample_file.iloc[:,1:-2].columns

    sample_file=sample_file.iloc[:,:-2]

    # 각 센서 데이터에 대해 z-변환 적용
    sample_file = sample_file.apply(z_normalize)
    sample_file=sample_file.iloc[:,7:]
    sample_file=sample_file.drop("Height",axis=1)
    sample_file=sample_file.drop("Proxi",axis=1)
    sample_file=sample_file.drop("Light",axis=1)
    sample_file=sample_file.drop("Step",axis=1)

    col_names=sample_file.columns
    ori_df=pd.concat([ori_df,sample_file],axis=0,ignore_index=True)
    mode=pd.concat([mode,sample_mode_file],
                      axis=0,ignore_index=True)

df=pd.DataFrame()
ori_df = ori_df.apply(z_normalize)

lst=[15,30 ,60,120,180,240,300]

for i in lst:
  df_mode=pd.DataFrame()
  df_mode = mode.iloc[::i].reset_index(drop=True)
  #for j in range(i,mode.shape[0]+1,i):
   # df_mode=pd.concat([df_mode,mode[j-1:j]],ignore_index=True)


#z_normalization

  df=ori_df
  df_columns=df.columns

# 1. DataFrame을 NumPy 배열로 변환
  df = df.values


  df= np.reshape(df, (-1, i, 12))
# df= np.reshape(df, (-1, 30, 12))
# df= np.reshape(df, (-1, 60, 12))
# df= np.reshape(df, (-1, 120, 12))
# df= np.reshape(df, (-1, 180, 12))
# df= np.reshape(df, (-1, 240, 12))

# 결과 확인
  X=df
  y=df_mode
  y=pd.get_dummies(y,columns=["Mode"])




  epochs=30
  learning_rates=0.001
  batch_size=128


    #데이터 분할
  X_train,X_test=train_test_split(X,test_size=0.3,random_state=42)
  y_train,y_test=train_test_split(y,test_size=0.3,random_state=42)



# 라벨을 one-hot 인코딩
#train_labels = to_categorical(y_train)
#test_labels = to_categorical(y_test)
#

# 2. DNN 모델 구축
  model = models.Sequential()
  model.add(layers.InputLayer(shape=(i,12)))
  model.add(layers.Flatten())
  model.add(layers.Dense(32, activation='relu'))  # 첫 번째 히든 레이어
  model.add(layers.Dense(64, activation='relu'))  # 두 번째 히든 레이어
  model.add(layers.Dense(128, activation='relu'))  # 세 번째 히든 레이어

  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(7, activation='softmax'))

# 3. 모델 컴파일
  model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. 모델 학습
  model.fit(X_train, y_train, epochs=30, batch_size=128)


# 5. 테스트 데이터로 성능 평가
  test_loss, test_acc = model.evaluate(X_test, y_test)
  print(f"{i/60}s의 Test accuracy: {test_acc}")






Mounted at /content/drive
Epoch 1/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 29s 2ms/step - accuracy: 0.6664 - loss: 0.8750
Epoch 2/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - accuracy: 0.8265 - loss: 0.4874
Epoch 3/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.8593 - loss: 0.4036
Epoch 4/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.8746 - loss: 0.3638
Epoch 5/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.8847 - loss: 0.3377
Epoch 6/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.8909 - loss: 0.3209
Epoch 7/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.8962 - loss: 0.3069
Epoch 8/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.8997 - loss: 0.2966
Epoch 9/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9027 - loss: 0.2891
Epoch 10/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.9055 - loss: 0.2810
Epoch 11/30
10474/10474 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - acc